In [1]:
%display latex

In [2]:
N = 3
F = GF(2^3, 'x')
x = F.gen()

In [3]:
basis = [x^3, x^5, x^6] # self-dual basis

In [4]:
# express an element as a linear combination of the basis
components = lambda k: [(k * el).trace() for el in basis]

# additive group character
chi = lambda k: exp(pi * I * int(k.trace()))

def toInt(k):
    return list(F).index(k)

In [5]:
y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

In [6]:
# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

In [7]:
# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

In [8]:
def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

In [9]:
for mu in F:
    curve = lambda t: mu * t # \beta = \mu \alpha
    sols = sign_perm(solve_basis(basis, curve), [1,1,1])
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
print('Recurrence relation holds!')

Recurrence relation holds!


In [10]:
def ray(mu):
    return lambda t: mu * t

In [11]:
def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [12]:
ps = PS(ray, [[1,1,1]] * 8)
ps

[ 1  1  1  1  1  1  1  1]
[ 1 -1 -I  1 -I  I  I -1]
[ 1 -I -1  I -I  I  1 -1]
[ 1  1  I  I  I  1  1  I]
[ 1 -I -I  I -1  1  I -1]
[ 1  I  I  1  1  I  1  I]
[ 1  I  1  1  I  1  I  I]
[ 1 -1 -1  I -1  I  I -I]

Now let's form the displacement operators.

In [13]:
def Proj(u, v=None):
    if not v:
        v = u
    return u.tensor_product(v.conjugate_tranpose())

Id = identity_matrix(2^N)

def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

In [21]:
def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

def D(a, b):
    return phi(a, b) * Z(a) * X(b)

The displacement operators should be trace orthogonal, null trace, unitary and hermititan.

In [22]:
# trace orthogonal
for a in F:
    for b in F:
        D1 = D(a, b)
        for c in F:
            for d in F:
                D2 = D(c, d)
                tr = (D1 * D2.conjugate_transpose()).trace()
                dl = int(a == c) * int(b == d)
                if tr != (2^N) * dl:
                    raise Exception

In [23]:
# null trace
for a in F:
    for b in F:
        if a != 0 and b != 0:
            d = D(a, b)
            if d.trace() != 0:
                raise Exception(a, b)

In [24]:
# hermitian
for a in F:
    for b in F:
        d = D(a, b)
        if d != d.conjugate_transpose():
            raise Exception

In [25]:
# unitarity
for a in F:
    for b in F:
        d = D(a,b)
        if d * d.conjugate_transpose() != Id:
            raise Exception

In [26]:
def graph(points):
    m = zero_matrix(SR, 2^N)
    for p in points:
        m[toInt(p[0]), toInt(p[1])] = 1
    return m

The property that we want to test is the following:
$$
D(\alpha_1,f(\alpha_1)) D(\alpha_2,f(\alpha_2))
= D(\alpha_1+\alpha_2, f(\alpha_1+\alpha_2))
= D(\alpha_1+\alpha_2, f(\alpha_1)+f(\alpha_2)),
$$
for a given curve $f$ and all $\alpha \in F$. Or for a set of points which gives a curve $\{\alpha,\beta\}$:
$$
D(\alpha_1,\beta_1) D(\alpha_2,\beta_2)
= D(\alpha_1+\alpha_2, \beta_1 + \beta_2).
$$

In [27]:
def testCurve(points):
    for p1 in points:
        for p2 in points:
            p_op = D(p1[0], p1[1]) * D(p2[0], p2[1])
            s_op = D(p1[0] + p2[0], p1[1] + p2[1]) # assuming additivity
            if p_op != s_op:
                raise Exception(p1, p2)
    return True

In [28]:
for mu in F:
    if not testCurve([(k, mu * k) for k in F]):
        raise Exception(mu)

The property holds for all the rays, as it should of course.

Now let's identify all of the family of curves that are valid for three qubits. But first let's test this example of an exceptional curve (equation 5.19 from Annals):
$$
(0,0), (\sigma^4,0), (\sigma^4, \sigma^5), (\sigma^3, \sigma^7), (\sigma^3, \sigma^4), (\sigma^6, \sigma^4), (\sigma^6, \sigma^7), (0, \sigma^5)
$$
This curve can be obtained by the equation:
$$
\beta^2 + \sigma^5 \beta = \sigma^6 \alpha + \sigma^2 \alpha^2.
$$

In [29]:
points = [(F(0),F(0)), (x^4,F(0)), (x^4,x^5), (x^3,x^7), (x^3,x^4), (x^6,x^4), (x^6,x^7), (F(0),x^5)]
points

[(0, 0),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x + 1, 1),
 (x + 1, x^2 + x),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (0, x^2 + x + 1)]

Just to be sure this curve is additive:

In [30]:
def checkAdditivity(points):
    for p1 in points:
        for p2 in points:
            p3 = (p1[0] + p2[0], p1[1] + p2[1])
            if p3 not in points:
                raise Exception
    return True

In [31]:
checkAdditivity(points)

True

In [34]:
graph(points)

[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0]

And now we test to see if the set of displacement operators are closed under our rule:

In [35]:
testCurve(points)

True

For example the points $(\sigma^6,\sigma^4)$ and $(\sigma^3,\sigma^7)$ give us the following operators:

In [48]:
phi(x^6,x^4)

I

In [52]:
D(x^6,x^4)

[ 0  0  0  0  I  0  0  0]
[ 0  0 -I  0  0  0  0  0]
[ 0  I  0  0  0  0  0  0]
[ 0  0  0  0  0  0  I  0]
[-I  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  I]
[ 0  0  0 -I  0  0  0  0]
[ 0  0  0  0  0 -I  0  0]

In [53]:
phi(x^3,x^7)

I

In [54]:
D(x^3,x^7)

[ 0  0  0  0  0  0  0  I]
[ 0  0  0  I  0  0  0  0]
[ 0  0  0  0  0  0 -I  0]
[ 0 -I  0  0  0  0  0  0]
[ 0  0  0  0  0 -I  0  0]
[ 0  0  0  0  I  0  0  0]
[ 0  0  I  0  0  0  0  0]
[-I  0  0  0  0  0  0  0]

Their product is:

In [55]:
D(x^6,x^4) * D(x^3,x^7)

[ 0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0 -1  0]
[ 0  0  0 -1  0  0  0  0]
[ 0  0 -1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1]
[ 1  0  0  0  0  0  0  0]
[ 0 -1  0  0  0  0  0  0]
[ 0  0  0  0  1  0  0  0]

And the operator given by the sum of their components is:

In [58]:
D(x^6+x^3,x^4+x^7)

[ 0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0 -1  0]
[ 0  0  0 -1  0  0  0  0]
[ 0  0 -1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1]
[ 1  0  0  0  0  0  0  0]
[ 0 -1  0  0  0  0  0  0]
[ 0  0  0  0  1  0  0  0]

And as we can see they are equal:

In [59]:
D(x^6,x^4) * D(x^3,x^7) == D(x^6+x^3,x^4+x^7)

True

In [46]:
# for p1 in points:
#     for p2 in points:
#         print(D(p1[0],p1[1]) * D(p2[0], p2[1]) - D(p1[0] + p2[0], p1[1] + p2[1]))

---
Now let's begin testing some other curves. Let's study the curve:
$$
\beta = \sigma^6 \alpha + \sigma^3 \alpha^2 + \sigma^5 \alpha^4.
$$

In [43]:
curve = lambda t: x^6 * t + x^3 * t^2 + x^5 * t^4
testCurve([(k, curve(k)) for k in F])

Exception: ((x^2, x^2 + x), (x + 1, x + 1))

In [60]:
list(F)

[0, x, x^2, x + 1, x^2 + x, x^2 + x + 1, x^2 + 1, 1]

In [63]:
p1 = (x^2,x^4)
p1

(x^2, x^2 + x)

In [64]:
p2 = (x^3,x^3)
p2

(x + 1, x + 1)

In [65]:
p3 = (p1[0] + p2[0], p1[1] + p2[1])
p3

(x^2 + x + 1, x^2 + 1)

In [67]:
D(*p1) * D(*p2)

[ 0  0  0  0  0  0 -1  0]
[ 0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0  0  1]
[ 0  0  0  0 -1  0  0  0]
[ 0  0  0 -1  0  0  0  0]
[ 0  1  0  0  0  0  0  0]
[-1  0  0  0  0  0  0  0]
[ 0  0  1  0  0  0  0  0]

In [69]:
D(*p3)

[ 0  0  0  0  0  0  1  0]
[ 0  0  0  0  0 -1  0  0]
[ 0  0  0  0  0  0  0 -1]
[ 0  0  0  0  1  0  0  0]
[ 0  0  0  1  0  0  0  0]
[ 0 -1  0  0  0  0  0  0]
[ 1  0  0  0  0  0  0  0]
[ 0  0 -1  0  0  0  0  0]

In [71]:
curve(x^2) == x^4

True

In [72]:
curve(x^3) == x^3

True